In [44]:
import numpy as np
import pandas as pd
import scipy as sp
import math
import itertools, random

Генерируем ковариационную матрицу:(моделируем данные (см. Аллен 4.4))

In [86]:
d=6 #размерность наблюдений
n=72 #число наблюдений
generating_mean=np.zeros(d)
covar_generating_matrix=[[1,0.3966,0.3688,0.1764,-0.4632,0.2939],[0.3966,1,0.0232,-0.0854,0.0193,0.2191],[0.3688,0.0232,1,0.0494,-0.1350,-0.2376],[0.1764,-0.0854,0.0494,1,-0.4671,0.1135],[-0.4632,0.0193,-0.1350,-0.4671,1,-0.3656],[0.2939,0.2191,-0.2376,0.1135,-0.3656,1]]
original_prec_matrix=np.linalg.inv(covar_generating_matrix)
print(np.linalg.det(covar_generating_matrix))
error_1_exp=0.05#ошибка первого типа для хи-квадрат теста
p0=0.2#вероятность обнуления элементов матрицы точности
non_p_d=0#сгенерированные не положительно определенные матрицы
w_s_c=0#Для сбора статистики о изначально неправильном детерминанте approx_prec_matrix
s_d_c=0#Для сбора статистики о изначально маленьких детерминантах approx_prec_matrix -- избегаем, чтобы погрешности не мещали
bad_addition=0#Для сбора статистики неправильной работы алгоритма, когда добавление ребра приводит к понижению логарифма правдоподобия
rep=10#число повторений эксперимента
full_graph=0#сколько раз все вершины были соединены одной компонентой связности
newton_descend_doesnt_work=0
actual_edges=np.zeros(rep)
empty_edges=np.zeros(rep)
false_edges=np.zeros(rep)

0.28236004008950344


In [87]:
for experiment_number in range (rep):
    print(experiment_number)
    rng=np.random.default_rng()
    actual_edges[experiment_number]=d*(d-1)/2
    generating_prec_matrix=original_prec_matrix.copy()
    for i in range(d):
        for j in range(d):
            if (i<j) and (rng.choice(2, p=[1-p0, p0])==1):
                generating_prec_matrix[i,j]=0
                generating_prec_matrix[j,i]=0
                actual_edges[experiment_number]-=1
    pos_def=1
    for i in range(d+1):
        if np.linalg.det(generating_prec_matrix[0:i, 0:i])<=0:
            pos_def=0
    if (pos_def==0):
        non_p_d+=1
        continue
    covar_generating_matrix=np.linalg.inv(generating_prec_matrix)
    data_sample=rng.multivariate_normal(generating_mean,covar_generating_matrix, size=n)
    d=data_sample.shape[1]
    n=data_sample.shape[0]
    sample_cov_matrix=np.cov(data_sample,rowvar=False)
    sample_prec_matrix=np.linalg.inv(sample_cov_matrix)
    sample_corr_matrix=np.corrcoef(data_sample, rowvar=False)
    graph=[]
    for i in range (d):
        graph.append((i,i)) #граф, в который будем вершины добавлять -- добавляем(i,i), потому что далее нужно будет множество ребер графа и пар(i,i)
    component_counter=1 #это чтобы знать, какой номер будет у новой компоненты связности, если мы создаем новую
    found_pair=1 #счетчик, который проверяет, что мы нашли хотя бы одну вершину
    #подбираем
    epsilon=0.0000000001
    approx_cov1_matrix=np.linalg.inv(np.diag(np.diag(np.linalg.inv(sample_cov_matrix)))) #это будет предыдущая по добавлению ребер ков. матрица, чтобы мы могли сравнить улучшение при добавлении каждого ребра
    while found_pair!=0:
        found_pair=0
        delta_max=False #счетчик, который гарантирует, что мы выбираем ребро с наибольшим модулем корреляции
        for a in range (d):
            for b in range (d):
                if ((a,b) not in graph) and a < b:
                    graph.append((a,b))
                    theta_a=[]#это цель итерационной процедуры -- найти матрицу, совпадающую (с учетом коэффициентов) с theta_a на элементах graph, и обратная которой равна 0 вне графа
                    for i in range (len(graph)):
                        if (graph[i][0]==graph[i][1]):
                            theta_a.append(-1/2*sample_cov_matrix[graph[i][0],graph[i][1]])
                        else:
                            theta_a.append(-sample_cov_matrix[graph[i][0],graph[i][1]])
                    theta_a=np.array(theta_a)
                    phi_i_a=[]#часть phi_i, которая соответсвует элементам graph
                    for i in range (len(graph)):
                        phi_i_a.append(sample_prec_matrix[graph[i][0],graph[i][1]])
                    phi_i_a=np.array(phi_i_a)
                    approx_prec_matrix=np.zeros((d,d))
                    for i in range (d):
                        for j in range (d):
                            if ((i,j) in graph) or((j,i) in graph):
                                approx_prec_matrix[i,j]=sample_prec_matrix[i,j]
                    while np.linalg.det(approx_prec_matrix)<0:#"выпрямляем" матрицу, если её детерминант стал отрицательным, умножая её на 0.1 и складывая с собственной диагональю до умножения
                        w_s_c+=1
                        approx_prec_matrix=np.diag(np.diag(approx_prec_matrix))+(approx_prec_matrix/10)
                        for i in range (d):
                            phi_i_a[i]=1.1*phi_i_a[i]
                        for i in range (len(graph)-d):
                            phi_i_a[d+i]=0.1*phi_i_a[d+i]
                    approx_cov_matrix=np.linalg.inv(approx_prec_matrix)
                    while np.linalg.det(approx_cov_matrix)<epsilon:
                        s_d_c+=1
                        approx_prec_matrix=approx_prec_matrix/2
                        for i in range (d):
                            phi_i_a[i]=phi_i_a[i]/2
                        for i in range (len(graph)-d):
                            phi_i_a[d+i]=phi_i_a[d+i]/2
                        approx_cov_matrix=np.linalg.inv(approx_prec_matrix)
                    theta_a_i=[]
                    for i in range (len(graph)):
                        if (graph[i][0]==graph[i][1]):
                            theta_a_i.append(-1/2*approx_cov_matrix[graph[i][0],graph[i][1]])
                        else:
                            theta_a_i.append(-approx_cov_matrix[graph[i][0],graph[i][1]])
                    theta_a_i=np.array(theta_a_i)
                    delta=100
                    epsilon=0.0000001
                    while (delta>epsilon):#по сути градиентный поиск точки --здесь шаг такой, что при линейной функции мы бы сразу попали в искомую точку. Аллен это называет методом Ньютона для решения неявных уравнений
                        #создаем матрицу Гамма_(1,1) -- составляем набор пар графа, считаем матрицу с ними, чтобы потом решить уравнение gamma_1_1_i*x=theta_a-theta_a_i и получить искомый сдвиг для phi_i_a
                        gamma_1_1_i=np.zeros((len(graph), len(graph)))
                        for i in range (len(graph)):
                            for j in range (len(graph)):
                                coef_mult=1
                                if (graph[i][0]==graph[i][1]):#коэффиценты отвечающие -1/2 у theta -- два минуса всегда обращаются в плюс, так что минусы игнорируем во всех случаях
                                    coef_mult/=2
                                if (graph[j][0]==graph[j][1]):
                                    coef_mult/=2
                                gamma_1_1_i[i,j]=coef_mult*(approx_cov_matrix[graph[i][0],graph[j][1]]*approx_cov_matrix[graph[i][1],graph[j][0]]+approx_cov_matrix[graph[i][0],graph[j][0]]*approx_cov_matrix[graph[i][1],graph[j][1]]) #спариваем элементы из i-й и j-й пары 2 способами
                        s=np.linalg.lstsq(gamma_1_1_i,theta_a-theta_a_i)[0] #предупреждение каждый раз вылезает, но оно вроде не важное
                        alpha=0.001
                        proper_step=False
                        while (proper_step==False):#проверяем, что шаг не слишком большой: детерминант не стал отрицательным
                            phi_i1_a=phi_i_a+alpha*s
                            #пересчитываем все итерационные величины по phi_i1_a
                            for i in range (len(graph)):
                                approx_prec_matrix[graph[i][0],graph[i][1]]=phi_i1_a[i]
                                approx_prec_matrix[graph[i][1],graph[i][0]]=phi_i1_a[i]
                            approx_cov_matrix=np.linalg.inv(approx_prec_matrix)
                            if np.linalg.det(approx_cov_matrix)>0:
                                proper_step=True
                            else:
                                alpha/=10
                                if (alpha<0.00000000001):
                                    proper_step='Broken'
                        if (proper_step=='Broken'):
                            newton_descend_doesnt_work+=1
                            break
                        delta=np.sum(np.square(phi_i1_a-phi_i_a))#меряем размер шага в качестве критерия остановки
                        phi_i_a=phi_i1_a
                        for i in range (len(graph)):
                            if (graph[i][0]==graph[i][1]):
                                theta_a_i[i]=-1/2*approx_cov_matrix[graph[i][0],graph[i][1]]
                            else:
                                theta_a_i[i]=-approx_cov_matrix[graph[i][0],graph[i][1]]
                    delta_k=n*(math.log(np.linalg.det(approx_cov1_matrix)) - math.log(np.linalg.det(approx_cov_matrix))) + (n-1)*(np.sum(sample_cov_matrix*(np.linalg.inv(approx_cov1_matrix)-np.linalg.inv(approx_cov_matrix))))
                    #проверка на значимость
                    if (delta_k<0):
                        bad_addition+=1
                    if (1-sp.stats.chi2.cdf(delta_k, 1) < 0.05/(d*(d+1)/2-(len(graph)-1))):
                        if (delta_max==False) or (delta_k > delta_max):
                            approx_cov1_matrix=approx_cov_matrix
                            found_pair=graph[-1]
                    graph.pop()
        graph.append(found_pair)        
    if len(graph)==d*(d+1)/2:
        full_graph+=1
    graph.pop()
    for i in range(len(graph)-d):
        if generating_prec_matrix[graph[i+d][0],graph[i+d][1]]==0:
            false_edges[experiment_number]+=1
    empty_edges[experiment_number]=actual_edges[experiment_number]-len(graph)+false_edges[experiment_number]+d

0


C:\Users\arkad\AppData\Local\Temp\ipykernel_8320\4111910697.py:93: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  s=np.linalg.lstsq(gamma_1_1_i,theta_a-theta_a_i)[0] #предупреждение каждый раз вылезает, но оно вроде не важное


1
2
3
4
5
6
7
8
9


In [88]:
actual_edges.mean()

12.3

In [89]:
empty_edges.mean()

8.5

In [90]:
((d*(d-1)/2)-actual_edges).mean()

2.7

In [91]:
false_edges.mean()

0.0

In [92]:
bad_addition

342

In [93]:
newton_descend_doesnt_work

0

In [94]:
non_p_d

0

In [95]:
w_s_c

1

In [96]:
s_d_c

0

In [97]:
full_graph

0

In [98]:
(empty_edges/actual_edges).mean()

0.6922760572760573

In [99]:
(false_edges/((d*(d-1)/2)-actual_edges)).mean()

0.0